In [1]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas

# Machine Learning
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [170]:
# Load the data into a Pandas DataFrame
d12_main_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
d12_main_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [172]:
# Generate summary statistics
d12_main_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [174]:
# Plot your data to see what's in your DataFrame
d12_main_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [176]:
d12_main_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, bitcoin to digibyte
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   price_change_percentage_24h   41 non-null     float64
 1   price_change_percentage_7d    41 non-null     float64
 2   price_change_percentage_14d   41 non-null     float64
 3   price_change_percentage_30d   41 non-null     float64
 4   price_change_percentage_60d   41 non-null     float64
 5   price_change_percentage_200d  41 non-null     float64
 6   price_change_percentage_1y    41 non-null     float64
dtypes: float64(7)
memory usage: 2.6+ KB


In [178]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler_data = StandardScaler().fit_transform(d12_main_data)

In [180]:
# Create a DataFrame with the scaled data
d12_main_data_scaled = pd.DataFrame(
        scaled_data,
        columns=d12_main_data.columns
)

# Copy the crypto names from the original DataFrame
d12_main_data_scaled["coin_id"] = d12_main_data.index

# Set the coin_id column as index
d12_main_data_scaled = d12_main_data_scaled.set_index("coin_id")

# Display the scaled DataFrame
d12_main_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [92]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [182]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
        model = KMeans(n_clusters=i, random_state=0)
        model.fit(d12_main_data_scaled)
        inertia.append(model.inertia_)

In [184]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
        "k": k,
        "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
d12_elbow = pd.DataFrame(elbow_data)

In [186]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_plot = d12_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)
elbow_plot

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Based on this data, the best value for 'k' is '4'.

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [191]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)

In [193]:
# Fit the K-Means model using the scaled DataFrame
model.fit(d12_main_data_scaled)

KMeans(n_clusters=4)

In [195]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
crypto_clusters = model.predict(d12_main_data_scaled)

# View the resulting array of cluster values.
print(crypto_clusters)

[0 0 3 3 0 0 0 0 0 3 3 3 3 0 3 0 3 3 0 3 3 0 3 3 3 3 3 3 0 3 3 3 1 0 3 3 2
 3 3 3 3]


In [197]:
# Create a copy of the scaled DataFrame
d12_main_data_scaled_predictions = d12_main_data_scaled.copy()

In [199]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
d12_main_data_scaled_predictions["crypto_cluster"] = crypto_clusters

# Display the copy of the scaled DataFrame
d12_main_data_scaled_predictions.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,crypto_cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,0
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,0
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,3
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,3
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,0


In [201]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
clusters_plot = d12_main_data_scaled_predictions.hvplot.scatter(
        x="price_change_percentage_24h",
        y="price_change_percentage_7d",
        by="crypto_cluster",
        hover_cols=["coin_id"],
        marker=["hex", "square", "cross", "inverted_triangle"],
        title="Cryptocurrencies Clusters",
)
clusters_plot

:NdOverlay   [crypto_cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [205]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [207]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
main_pca_data = pca.fit_transform(d12_main_data_scaled)

# View the scaled PCA data
main_pca_data[:5]

array([[-0.60066733,  0.84276006,  0.46159457],
       [-0.45826071,  0.45846566,  0.95287678],
       [-0.43306981, -0.16812638, -0.64175193],
       [-0.47183495, -0.22266008, -0.47905316],
       [-1.15779997,  2.04120919,  1.85971527]])

In [209]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.3719856 , 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the the three principal components is 88.81% after adding them together.

In [212]:
# Create a new DataFrame with the PCA data.
d12_main_data_pca = pd.DataFrame(
        main_pca_data,
        columns=["PC1", "PC2", "PC3"]
)

# Copy the crypto names from the original scaled DataFrame
d12_main_data_pca["coin_id"] = d12_main_data.index

# Set the coin_id column as index
d12_main_data_pca = d12_main_data_pca.set_index("coin_id")

# Display the scaled PCA DataFrame
d12_main_data_pca.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600667,0.842760,0.461595
ethereum,-0.458261,0.458466,0.952877
tether,-0.433070,-0.168126,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157800,2.041209,1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [144]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [214]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
        model = KMeans(n_clusters=i, random_state=0)
        model.fit(d12_main_data_pca)
        inertia.append(model.inertia_)

In [216]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
        "k": k,
        "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
d12_elbow_pca = pd.DataFrame(elbow_data_pca)

In [218]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = d12_elbow_pca.hvplot.line(x="k", y="inertia", title="Elbow Curve Using PCA Data", xticks=k)
elbow_plot_pca

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for `k` is `4`.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** It does not differ at all. Both are identical

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [222]:
# Initialize the K-Means model using the best value for k
model_pca = KMeans(n_clusters=4)

In [224]:
# Fit the K-Means model using the PCA data
model_pca.fit(d12_main_data_pca)

KMeans(n_clusters=4)

In [226]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
crypto_clusters_pca = model_pca.predict(d12_main_data_pca)

# Print the resulting array of cluster values.
print(crypto_clusters_pca)

[0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 3 0 0 2 0 0 3 1
 0 0 0 0]


In [228]:
# Create a copy of the scaled PCA DataFrame
d12_main_data_pca_predictions = d12_main_data_pca.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
d12_main_data_pca_predictions["crypto_cluster"] = crypto_clusters_pca

# Display the copy of the scaled PCA DataFrame
d12_main_data_pca_predictions.head()

,PC1,PC2,PC3,crypto_cluster
coin_id,,,,
bitcoin,-0.600667,0.842760,0.461595,0
ethereum,-0.458261,0.458466,0.952877,0
tether,-0.433070,-0.168126,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157800,2.041209,1.859715,0


In [230]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
clusters_plot_pca = d12_main_data_pca_predictions.hvplot.scatter(
        x="PC1",
        y="PC2",
        by="crypto_cluster",
        hover_cols=["coin_id"],
        marker=["hex", "square", "cross", "inverted_triangle"],
        title = "Cryptocurrencies Clusters Using PCA Data",
)
clusters_plot_pca

:NdOverlay   [crypto_cluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [233]:
# Composite plot to contrast the Elbow curves
elbow_plot + elbow_plot_pca

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [235]:
# Composite plot to contrast the clusters
clusters_plot + clusters_plot_pca

:Layout
   .NdOverlay.I  :NdOverlay   [crypto_cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [crypto_cluster]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The reduced features will still show similar results to the original model and make it easier to understand the cluster visualizations.